In [1]:
# Importa as bibliotecas
import pandas as pd
import os.path

ssesso = 'QLQR123'
DEALER = 'sim'
path_export = r'caminho\pasta\basebi'

In [2]:
# Leitura das variáveis
df_var = pd.read_excel('caminho\pasta\excelntratado', 
                       sheet_name='namekk', dtype={'Variable': str, 'Value': str})

df_var = df_var.transpose()
df_var.columns = df_var.iloc[0]
df_var = df_var[1:]

# Mês de referência para cálculo do yrtodt
prior_year = int(df_var['prior_year'].iloc[0])
calendar_year = int(df_var['calendar_year'].iloc[0])
MES_ATUAL = int(df_var['MES_ATUAL'].iloc[0])

# Definição das pastas e dos arquivos
pasta_output = str(df_var['pasta_output'][0])
pasta_reporte = str(df_var['pasta_reporte'][0])
pasta_raiz = str(df_var['pasta_raiz'][0])
pasta_PBI = str('caminho\pasta\powerbi')
path_not_assigned = r'caminho\pasta\not assigned'

# Define as demais pastas do sistema
pasta_celaut = pasta_raiz + '1. Dados sell out'

In [3]:
MES_ATUAL

9

In [ ]:
df_lm = pd.read_excel(pasta_celaut + rf'\loja\lojayrtodt{MES_ATUAL}.xlsx', header=0, sheet_name=str('LDM'))
df_lm.head(3)

In [ ]:
df_catalogo = pd.read_excel(pasta_celaut + r'\loja\catalogoldm.xlsx',usecols=['Cód. Item Fornecedor', 'BU', 'Segmento', 'Cordless'])
df_catalogo.head(3)

In [ ]:
df_lm = pd.merge(df_lm, df_catalogo, how='left')
df_lm.tail(3)

In [7]:
def ajustar_ultimos_digitos(x):
    x = str(x)
    unittokk = 'Not assigned'
    
    if len(x)>=10:
        if x[-4::]=='-000':
            unittokk = x[:-4]
        elif len(x) == 10:
            unittokk = str(x)
        else:
            x = 'Not assigned'
    else:
        x = 'Not assigned'
    
    return unittokk

In [ ]:
# Tratamento das variáveis
df_lm['Units'] = df_lm['Qtd. 2021'].fillna(0).astype(int)
df_lm['Sales'] = df_lm['Fat. 2021'].fillna(0).astype(float)

df_lm['Cód. Item Fornecedor'] = df_lm['Cód. Item Fornecedor'].replace('0', 'Not assigned')
df_lm['unittokk'] = df_lm['Cód. Item Fornecedor'].apply(lambda x: str(x).replace('.','').replace('-000', ''))
df_lm['unittokk'] = df_lm['unittokk'].apply(lambda x: ajustar_unittokk(x))

df_lm['Product'] = df_lm['Product'].astype(str)
df_lm['Year'] = df_lm['Data'].apply(lambda x: int(str(x)[0:4]))
df_lm['Month'] = df_lm['Data'].apply(lambda x: int(str(x)[5:7]))
df_lm['ssesso'] = ssesso
df_lm['Type'] = 'Online'
df_lm['Dealer'] = DEALER
df_lm['Segment'] = df_lm['Segmento']
df_lm['Stock'] = 0
df_lm['Place'] = 'Online'

df_lm_clean = df_lm[['ssesso', 'Dealer', 'Place', 'Type', 'Product', 'unittokk', 'BU', 'Segment', 'Cordless', 'Year', 'Month', 'Units', 'Sales', 'Stock']]

df_lm_clean['unittokk'].fillna('Not assigned',inplace=True)
df_lm_clean['Product'].fillna('Not assigned',inplace=True)
df_lm_clean['Sales'].fillna(0,inplace=True)
df_lm_clean['Units'].fillna(0,inplace=True)
df_lm_clean['Stock'].fillna(0,inplace=True)
df_lm_clean['Segment'].fillna('Not assigned',inplace=True)
df_lm_clean['Cordless'].fillna('Not assigned',inplace=True)
df_lm_clean['BU'].fillna('Not assigned',inplace=True)
df_lm_clean.tail(3)

In [ ]:
df_faltantes = df_lm[df_lm['unittokk']=='Not assigned']
df_faltantes.reset_index(drop=True , inplace=True)
df_faltantes.drop_duplicates(subset=['Cód. Produto', 'EAN'],inplace=True)
df_faltantes.reset_index(drop=True, inplace=True)
df_faltantes = df_faltantes[['Cód. Produto', 'EAN', 'Product']]
df_faltantes.to_excel(path_not_assigned+r'\not assigned da loja.xlsx', index=False)



In [10]:
teste = df_lm_clean.query("Year == 2023 & Month == 9")
print(teste['Sales'].sum())

1899038.1300000001


In [11]:
df_lm_clean.to_csv(r'caminho\do\csv', sep=';', decimal=',', index=False, encoding="utf-8-sig")